# Pipeline completo de limpieza de datos

## Importar bibliotecas

In [1]:
import pandas as pd
import numpy as np

print('Versiones:')
print(f'  - pandas: {pd.__version__}')
print(f'  - numpy:  {np.__version__}')

Versiones:
  - pandas: 2.3.3
  - numpy:  2.3.4


## Crear dataset con problemas comunes

In [2]:
datos = {
    'id': [1, 2, 3, 4, 5, 1, 6],  # Duplicado en id 1
    'nombre': ['Ana García', 'Carlos López', 'María Rodríguez', 'Juan Pérez', 'Ana García', 'ana garcia', 'Luis Martín'],
    'edad': ['25', '30', '28', '35', '25', '25', '40'],  # String en lugar de int
    'email': ['ana@email.com', 'carlos@email.com', 'maria@email.com', 'juan@email.com', 'ana@email.com', 'ana@email.com', 'luis@email.com'],
    'salario': [45000, 55000, 48000, 60000, 45000, 45000, 52000],
    'departamento': ['Ventas', 'IT', 'Marketing', 'IT', 'ventas', 'VENTAS', 'Recursos Humanos']  # Inconsistente capitalización
}

df = pd.DataFrame(datos)
print("Datos originales con problemas:")
print(df)

Datos originales con problemas:
   id           nombre edad             email  salario      departamento
0   1       Ana García   25     ana@email.com    45000            Ventas
1   2     Carlos López   30  carlos@email.com    55000                IT
2   3  María Rodríguez   28   maria@email.com    48000         Marketing
3   4       Juan Pérez   35    juan@email.com    60000                IT
4   5       Ana García   25     ana@email.com    45000            ventas
5   1       ana garcia   25     ana@email.com    45000            VENTAS
6   6      Luis Martín   40    luis@email.com    52000  Recursos Humanos


## Inspeccionar y diagnosticar problemas

In [3]:
print(f"\nTipos de datos: {df.dtypes}")
print(f"\nDuplicados por id: {df['id'].duplicated().sum()}")
print(f"Duplicados completos: {df.duplicated().sum()}")
print(f"\nValores únicos en departamento: {df['departamento'].unique()}")


Tipos de datos: id               int64
nombre          object
edad            object
email           object
salario          int64
departamento    object
dtype: object

Duplicados por id: 1
Duplicados completos: 0

Valores únicos en departamento: ['Ventas' 'IT' 'Marketing' 'ventas' 'VENTAS' 'Recursos Humanos']


## Limpiar duplicados

In [4]:
# Eliminar duplicados basados en id y email

#df_limpio = df.drop_duplicates(subset=['id', 'email'], keep='first')
# modificación para evitar futuros SettingWithCopyWarning
df_limpio = df.drop_duplicates(subset=['id', 'email'], keep='first').copy()
print(f"\nDespués de eliminar duplicados: {len(df_limpio)} filas")


Después de eliminar duplicados: 6 filas


## Corregir tipos de datos y formatos

In [5]:
# Convertir edad a numérico
df_limpio['edad'] = pd.to_numeric(df_limpio['edad'], errors='coerce')

# Normalizar departamento
df_limpio['departamento'] = df_limpio['departamento'].str.title()

# Normalizar nombres
df_limpio['nombre'] = df_limpio['nombre'].str.title()

print("Después de correcciones:") 
print(df_limpio) 
print(f"\nTipos corregidos: {df_limpio.dtypes}")

Después de correcciones:
   id           nombre  edad             email  salario      departamento
0   1       Ana García    25     ana@email.com    45000            Ventas
1   2     Carlos López    30  carlos@email.com    55000                It
2   3  María Rodríguez    28   maria@email.com    48000         Marketing
3   4       Juan Pérez    35    juan@email.com    60000                It
4   5       Ana García    25     ana@email.com    45000            Ventas
6   6      Luis Martín    40    luis@email.com    52000  Recursos Humanos

Tipos corregidos: id               int64
nombre          object
edad             int64
email           object
salario          int64
departamento    object
dtype: object


## Crear columnas calculadas

In [6]:
# Calcular salario mensual y anual

df_limpio['salario_mensual'] = df_limpio['salario'] / 12
df_limpio['categoria_edad'] = pd.cut(df_limpio['edad'], 
                                    bins=[0, 25, 35, 100], 
                                    labels=['Joven', 'Adulto', 'Senior'])

print("Con columnas calculadas:")
print(df_limpio[['nombre', 'edad', 'categoria_edad', 'salario', 'salario_mensual']])

Con columnas calculadas:
            nombre  edad categoria_edad  salario  salario_mensual
0       Ana García    25          Joven    45000      3750.000000
1     Carlos López    30         Adulto    55000      4583.333333
2  María Rodríguez    28         Adulto    48000      4000.000000
3       Juan Pérez    35         Adulto    60000      5000.000000
4       Ana García    25          Joven    45000      3750.000000
6      Luis Martín    40         Senior    52000      4333.333333


## Inspección final y última limpieza de datos

Una vez efectuadas la eliminación de duplicados, correccción de tipos de datos y formatos, y la creación de columnas calculadas, se visualizan los siguientes problemas:
- 1 fila duplicada generada luego de la normalización de los datos de las columnas "nombre" y "departamento", y
- formato incorrecto para el salario mensual, el que parece estar en US$, por lo que lo correcto sería considerar 2 decimales.

In [7]:
# últimas correcciones a los datos

# redondear el salario_mensual a 2 cifras decimales (moneda USS)
df_limpio['salario_mensual'] = df_limpio['salario_mensual'].round(2)

# eliminar duplicados in situ
df_limpio.drop_duplicates(subset=['email'], keep='first', inplace=True)
df_limpio.reset_index(drop=True, inplace=True)

print('\nDespués de limpieza final:\n')
print(f'Tipos de datos: {df_limpio.dtypes}\n')
print(f'Dimensión de df_limpio = {df_limpio.shape}\n')
print(df_limpio)      


Después de limpieza final:

Tipos de datos: id                    int64
nombre               object
edad                  int64
email                object
salario               int64
departamento         object
salario_mensual     float64
categoria_edad     category
dtype: object

Dimensión de df_limpio = (5, 8)

   id           nombre  edad             email  salario      departamento  \
0   1       Ana García    25     ana@email.com    45000            Ventas   
1   2     Carlos López    30  carlos@email.com    55000                It   
2   3  María Rodríguez    28   maria@email.com    48000         Marketing   
3   4       Juan Pérez    35    juan@email.com    60000                It   
4   6      Luis Martín    40    luis@email.com    52000  Recursos Humanos   

   salario_mensual categoria_edad  
0          3750.00          Joven  
1          4583.33         Adulto  
2          4000.00         Adulto  
3          5000.00         Adulto  
4          4333.33         Senior  


In [8]:
print('La limpieza del dataset original, df, ha teminado:')
print(f'  - {len(df) - len(df_limpio)} filas duplicadas fueron eliminadas')
print('  - Los datos fueron normalizados')
columnas_nuevas = [col for col in df_limpio.columns if col not in df.columns]
print(f'  - {df_limpio.shape[1] - df.shape[1]} columnas calculadas fueron creadas: {columnas_nuevas}')

La limpieza del dataset original, df, ha teminado:
  - 2 filas duplicadas fueron eliminadas
  - Los datos fueron normalizados
  - 2 columnas calculadas fueron creadas: ['salario_mensual', 'categoria_edad']
